In [1]:
import torch
from torch.nn.functional import one_hot
from bayesian_network.common.torch_settings import TorchSettings
import torchvision
from torchvision.transforms import transforms

gamma = 0.00001
torch_device = torch.device("cpu")
dtype = torch.float64
selected_num_observations = 1000

torch_settings = TorchSettings(torch_device, dtype)

mnist = torchvision.datasets.MNIST('./mnist', train=True, transform=transforms.ToTensor(), download=True)
data = mnist.train_data.to(torch_settings.device)
data = data.ge(128).long()

height, width = data.shape[1:3]
num_features = height * width
num_observations = data.shape[0]

# Morph into evidence structure
training_data_reshaped = data.reshape([num_observations, num_features])

# Make smaller selection
if selected_num_observations:
    training_data_reshaped = training_data_reshaped[0:selected_num_observations]
    num_observations = training_data_reshaped.shape[0]

# evidence: List[num_observed_nodes x torch.Tensor[num_observations x num_states]], one-hot encoded
gamma = 0.000001
evidence = [
    node_evidence * (1-gamma) + gamma/2
    for node_evidence 
    in one_hot(training_data_reshaped.T, 2).to(torch_settings.dtype)
]
        
evidence

/Users/pimmostert/Repositories/BN_experiments/.venv/lib/python3.10/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


[tensor([[1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         ...,
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07]], dtype=torch.float64),
 tensor([[1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         ...,
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07]], dtype=torch.float64),
 tensor([[1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         ...,
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07]], dtype=torch.float64),
 tensor([[1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         ...,
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07],
         [1.0000e+00, 5.0000e-07]], dtype=torch.float64),
 tensor([[1.